In [8]:
import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
import time

from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader, sampler
from torch import nn
from train import acc_metric, dice_metric
from DatasetLoader import DatasetLoader
from Unet2D import Unet2D

In [24]:
def test(model, test_dl, acc_fn, dice_fn):
    model.cuda()
    model.eval()
    running_acc = 0
    running_dice = 0
    for x, y in test_dl:
        x = x.cuda()
        y = y.cuda()
        with torch.no_grad():
            outputs = model(x)
            acc = acc_fn(outputs, y)
            dice = dice_fn(outputs, y)
        running_acc += acc
        running_dice += dice
    
    return running_acc/len(test_dl), running_dice/len(test_dl)

In [25]:
def main(params_path):
    bs=1
    
    # load the test data
    base_path = Path('/work/datasets/medical_project/CAMUS_resized')
    data = DatasetMedical(base_path / 'train_gray',
                         base_path / 'train_gt')
    print(len(data))

    # split the training dataset and initialize the data loaders
    _, _, test_dataset = torch.utils.data.random_split(
        data,
        (300, 100, 50),
        generator=torch.Generator().manual_seed(42)
    )

    # split the training dataset and initialize the data loaders
    test_data = DataLoader(test_dataset, batch_size=bs, shuffle=False)
    print(len(test_data))
    xb, yb = next(iter(test_data))
    print(xb.shape, yb.shape)

    # build the Unet2D with one channel as input and 2 channels as output
    unet = Unet2D(1, 2)
    # Load best model params
    unet.load_state_dict(torch.load(params_path))
    # loss function and optimizer
    # loss_fn = nn.CrossEntropyLoss()
    # opt = torch.optim.Adam(unet.parameters())

    # do some training
    acc, dice = test(unet, test_data, acc_metric, dice_metric)
    print('acc: ', acc,'dice: ', dice)

In [22]:
params_path = "models/model_epoch_41.pth"
main(params_path)

450
50
torch.Size([1, 1, 384, 384]) torch.Size([1, 384, 384])
acc:  tensor(0.9822, device='cuda:0') dice:  tensor(0.8568, device='cuda:0')
